In [ ]:
import sys
sys.path.append(r'C:\Users\ablack.GAMRY\Documents\Jupyter\Supporting\Release')
from toolkitcommon import *
from toolkitcurves import *
import UniversalGraph as Graph
import numpy as np                       #Used to store and manipulate raw data output
import time                              #Used for script time delay
import pandas as pd                      #Used for presenting data in tables
import matplotlib.pyplot as plt          #Used to plot data
from scipy import stats                  #Used for linear regression
import csv                               #Used to export data as csv files
import traceback                         #Used for error catching
import math
from matplotlib import pyplot as plt     #Used for plotting
%matplotlib notebook 
#matplolib notebook activates QT GUI for dynamic plots
from mpl_point_clicker import clicker  

toolkitpy_init("RunPyBind")
Pstat = PyPstat("PSTAT")
Pstat.set_ctrl_mode(PSTATMODE)

FileName = "CorrectName" #Parent file name. Output data files will be named "file_name with x scan rate.csv"
VInit = 0.5 #Initial Voltage
VLimit1 = -0.25 #Potentiostat scans to Voltage Apex 1
V1hold = 0.0 #Hold time for first vertex (s)
VLimit2 = 0.5 #Potentiostat scans to Voltage Apex 2
V2hold = 0.0 #Hold time for second vertex (s)
VFinal = 0.5 #Final Voltage
Vfhold = 0.0 #Hold time for the final vertex (s)
ScanRate = 0.1 #Scan rate (V/s)
ResVal  = 0.001 #Step Size (V)
Cycles = 1 #Number of cycles per scan
NumOfScans = 5   #Total number of scans collected sequentially
Increment = 0.01 #Between each scan from NumOfScans, the ScanRate will change by Increment (V/s)
ScanDelay = 55    #Time delay between each subsequent scan. (s)
SampleTime  = ResVal/ScanRate
#This one will be a bit harder maybe and EIS will be the hardest

try:
    fig = plt.figure()
    ax = fig.gca()
    
    SampleMode = 1 
    
    ScanRates = []
    data_frame = dict()
    for ScanNumber in range(NumOfScans):
        Signal = Pstat.signal_r_up_dn_new([VInit, VLimit1, VLimit2, VFinal], [ScanRate, ScanRate, ScanRate], 
                                       [V1hold,V2hold,Vfhold],SampleTime,Cycles,SampleMode)
        Pstat.set_signal_r_up_dn(Signal)
        Pstat.init_signal()
        Pstat.set_cell(True)
        MaxSize = 100000
        CV = RCVWrapper(Pstat,MaxSize)
        CV.run(True)
        fileName = FileName + ' with ' +str(ScanRate) + ' scan rate.csv'
        while CV.running():
            time.sleep(0.010)
            data = CV.acq_data()
            np.savetxt(fileName, data, delimiter = ',', header = 'IERange, Overload, Time, Vsig, Ach, Im, Vf, Vu, Cycle, Temp, StopTest')
            if not data.size == 0:
                Graph.plotCV(CV, data, ScanRate,ScanRates, ScanNumber, ScanDelay, data_frame,fig, ax)
            
        data_frame[ScanNumber] = data #data is added to the dictionary
        ScanRates.append(ScanRate) #the ScanRate used for data collection is added to a list
        ScanRate = round(ScanRate + Increment, 2) #ScanRate is changed by Increment for the next iteration of the for loop
        Pstat.set_cell (True) #cell is turned off to prepare for the next interation of the loop
        time.sleep(ScanDelay) #Scan delay between CV interations as determined by user parameter input     
        
except BaseException as err:
    
    print(err)
    traceback.print_exception(*sys.exc_info())
    #Pstat.SetCell (CELLSTATE.OFF)
    raise
Pstat.set_cell(False)